In [1]:
import sqlite3
import csv

# Correspondance nom scientifique - nom commun

In [2]:
# Tableau de correspondance Nom latin, Nom commun :
nomlatin_csv = 'data/nomlatin_lesarbresfr.csv'

with open(nomlatin_csv, 'r') as f:
    reader = csv.reader(f)
    nomslatins = { row[1]: row[0] for row in reader }
    
    # Remarque: il y a parfois plus de 3 colonnes 

In [3]:
# Lecture des triplets ( genre, espece, variete  ) :

db_filename = 'data/arbres.db'
db = sqlite3.connect(db_filename)


cursor = db.cursor()

cursor.execute("""
SELECT count(*) AS c, genre_bota, espece, variete FROM arbres
GROUP BY genre_bota, espece, variete
ORDER BY genre_bota
             """)

#nametuples = cursor.fetchmany(10)
nametuples = cursor.fetchall()

In [49]:
import unidecode

In [50]:
def filterlist( text, liste, startswith=False ):
    """ retourne une nouvelle liste ne comprenant que les élements (string)
        dans lesquels est présent text (string)
        
        startswith=True : force 'text' à être present au début
    """
    # case insensitive
    text = text.lower()
    liste = [ m.lower() for m in liste ]
    # no accent
    text = unidecode.unidecode( text )
    liste = [ unidecode.unidecode(m) for m in liste ]
    
    if startswith:
        crible = lambda A, B: B.startswith( A )
    else:
        crible = lambda A, B: A in B

    
    filtered_list = [ ele for ele in liste if crible( text, ele ) ]
    return filtered_list

In [51]:
filterlist( 'u', ['u', 'ui', 'dfqU', 'r'], startswith=True )

['u', 'ui']

In [54]:
# def keep_commonpart

def keep_only_commonpart( liste ):
    """ ne garde que la partie présente dans toute les mots de la 'liste'
    """
    
    N = min( [len(mot) for mot in liste] )

    common = []
    for i in range(N):
        if len( { mot[i] for mot in liste } ) == 1:
            common.append( liste[0][i] )
        else:
            break

    common = ''.join( common ).strip(' ')

    return common

In [55]:
def findmatch( triplet ):
    genre, espece, variete = triplet
        
    nomspossibles = list( nomslatins.keys() )
    
    # filter
    if genre:
        nomspossibles = filterlist( genre, nomspossibles, startswith=True )
   
    if espece:
        nomspossibles = filterlist( espece, nomspossibles )
        
    if variete and len(nomspossibles)>1 :
        nomspossibles = filterlist( variete, nomspossibles )
        
    tuple_str = ' '.join([ str(r) if r else '-' for r in row[1:]])
    #print( tuple_str, len( nomspossibles ) )
    
    if len(nomspossibles) == 1:
        match = nomspossibles[0]
    elif len(nomspossibles) > 1:
        match = keep_only_commonpart( nomspossibles )
        if match : match += '*'
    else:
        match = False
        
    return match

In [56]:
identified = []
non_identified = []
for row in nametuples:
   
    match = findmatch( row[1:] )
    if match:
        identified.append(( row, match ))
    else:
        non_identified.append( row )
        
print( len(identified), 'trouvés' )
print( len(nametuples), 'total' )

294 trouvés
547 total


non unique name...

In [57]:
sorted( non_identified, key=lambda x:x[0], reverse=True )

[(783, None, None, None),
 (776, 'Liquidambar', 'styraciflua', None),
 (706, 'Betula', 'verrucosa', None),
 (423, 'Alnus', 'cordata', None),
 (414, 'Sophora', 'japonica', None),
 (336, 'Pyrus', 'calleryana', 'Chanticleer'),
 (331, 'Acer', 'saccharinum', 'Wierii'),
 (269, 'Koelreuteria', 'paniculata', None),
 (223, 'Populus', 'robusta', None),
 (160, 'Carpinus', None, 'Pyramidalis'),
 (153, 'Acer', 'cappadocicum', None),
 (142, 'Ulmus', 'minor', None),
 (110, 'Ginkgo', 'biloba', None),
 (99, 'Fraxinus', 'ornus', None),
 (97, 'Fraxinus', None, 'Flame'),
 (95, 'Betula', 'alba', None),
 (89, 'Gleditsia', 'triacanthos', 'Inermis'),
 (87, 'Catalpa', 'bignonoides', None),
 (79, 'Platanus', 'platanor', 'Vallis clausa'),
 (79, 'Quercus', 'robur', None),
 (77, 'Tetradium', 'danielli', None),
 (72, 'Cedrus', 'deodara', None),
 (70, 'Albizia', 'jullibrissin', None),
 (70, 'Alnus', 'spaethii', None),
 (69, 'Fraxinus', 'oxycarpa', 'Flame'),
 (65, 'Metasequoia', 'glyptostroboides', None),
 (53, 'Amél

In [9]:
db.close()

In [ ]:
# Tableau de correspondance Nom latin, Nom commun :
nomlatin_csv = 'data/nomlatin_lesarbresfr.csv'

with open(nomlatin_csv, 'r') as f:
    reader = csv.reader(f)
    nomslatins = { row[1]: row[0] for row in reader }
    
    # Remarque: il y a parfois plus de 3 colonnes 

In [ ]:
nomslatins

In [ ]:
for row in reader:
    print( row )